# Falabella

In [1]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
df_ml.dropna(subset=["tweet_text"], inplace=True)
df_ml = df_ml[df_ml["sentiment"] != "others"]
df_ml = df_ml.sample(n=100)
# df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
# df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")

In [2]:
def df_to_list(df):
    texts = df["tweet_text"]
    data = []
    for text in texts:
        data.append(text)
    return data

In [3]:
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
tokenizer_español = BertTokenizer.from_pretrained("../pytorch/", do_lower_case=False)
model = BertModel.from_pretrained("../pytorch")

Some weights of the model checkpoint at ../pytorch were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from src import Features
inputs = df_to_list(df_ml)
inputs = Features.bert_encoder(inputs, tokenizer_español)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
data_ml = {
        "max_polling_std": Features.std_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_z_score": Features.z_score_normalization(Features.max_polling(last_hidden_states)),
        "max_polling_max": Features.max_min_normalization(Features.max_polling(last_hidden_states)),
        "mean_polling_std": Features.std_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_z_score": Features.z_score_normalization(Features.mean_polling(last_hidden_states)),
        "mean_polling_max": Features.max_min_normalization(Features.mean_polling(last_hidden_states)),
    }

In [5]:
def find_indexs(list, value):
    list_aux = list.copy()
    output = []
    while True:
        try:
            index = list_aux.index(value)
            output.append(index)
            list_aux[index] = None
        except ValueError:
            break
    return output

# agrupamiento

# k-means

In [6]:
# k-means
from sklearn.cluster import KMeans
from sklearn import metrics
agrupamiento = {}
metric = []
for i in range(100):
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=4,
            init="random",
            max_iter=100,
            random_state=i,
            n_init=1,
        ).fit(value)
        agrupamiento[key] = list(kmeans.labels_)
        metrica.append(metrics.silhouette_score(
            value, kmeans.labels_, metric="sqeuclidean"))
    metric.append(metrica)
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs = []
for i in range(6):
    list_aux = []
    for metrica in metric:
        list_aux.append(metrica[i])
    max_ = max(list_aux)
    index = list_aux.index(max_)
    rs.append([index, round(max_, 3)])
rs

[[26, 0.371], [27, 0.082], [18, 0.544], [79, 0.202], [89, 0.223], [32, 0.239]]

In [7]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="random",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i=0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rs rsScore cluster 1 cluster 2 cluster 3 cluster 4
26 & 0.371 & anger (6), sadness (3), joy (1) & sadness (2) & anger (70), sadness (14), joy (2) & joy (1), sadness (1)
27 & 0.082 & sadness (6), anger (3), joy (2) & anger (72), sadness (12), joy (2) & sadness (2) & anger (1)
18 & 0.544 & anger (18), sadness (6), joy (2) & anger (31), sadness (10), joy (1) & joy (1), sadness (1) & anger (27), sadness (3)
79 & 0.202 & anger (69), sadness (13), joy (2) & anger (5), sadness (4) & anger (2), joy (2) & sadness (3)
89 & 0.223 & anger (6), sadness (5), joy (1) & anger (66), sadness (12), joy (1) & joy (2) & anger (4), sadness (3)
32 & 0.239 & anger (3), sadness (1), joy (1) & anger (37), sadness (11) & anger (35), sadness (7), joy (3) & anger (1), sadness (1)


# k-means++

In [17]:
agrupamiento = {}
metric = []
for i in range(100):
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=4,
            init="k-means++",
            max_iter=100,
            random_state=i,
            n_init=1,
        ).fit(value)
        agrupamiento[key] = list(kmeans.labels_)
        metrica.append(metrics.silhouette_score(
            value, kmeans.labels_, metric="sqeuclidean"))
    metric.append(metrica)
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs = []
for i in range(6):
    list_aux = []
    for metrica in metric:
        list_aux.append(metrica[i])
    max_ = max(list_aux)
    index = list_aux.index(max_)
    rs.append([index, round(max_, 3)])
rs


[[22, 0.514], [57, 0.085], [51, 0.548], [82, 0.147], [82, 0.127], [64, 0.243]]

In [18]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="k-means++",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rs rsScore cluster 1 cluster 2 cluster 3 cluster 4
22 & 0.514 & anger (75), sadness (19), joy (3) & sadness (1) & anger (1) & joy (1)
57 & 0.085 & sadness (2), anger (2), joy (1) & anger (71), sadness (16), joy (3) & anger (3), sadness (1) & sadness (1)
51 & 0.548 & anger (34), sadness (10), joy (1) & joy (1), sadness (1) & anger (20), sadness (6), joy (2) & anger (22), sadness (3)
82 & 0.147 & anger (24), sadness (6), joy (2) & anger (48), sadness (9), joy (1) & anger (1), joy (1), sadness (1) & sadness (4), anger (3)
82 & 0.127 & anger (27), sadness (12), joy (2) & anger (48), sadness (7), joy (1) & anger (1), sadness (1) & joy (1)
64 & 0.243 & anger (44), sadness (12) & anger (31), sadness (8), joy (3) & anger (1) & joy (1)


# fuzzy c-means

In [10]:
from fcmeans import FCM
agrupamiento = {}
metric = []

rs = []
for key, value in data_ml.items():
    data = np.array(value)
    fcm = FCM(n_clusters=4)
    fcm.fit(data)
    labels = fcm.predict(data)
    grupos = fcm.soft_predict(data)
    agrupamiento[key] = labels
    rs.append(round(metrics.silhouette_score(
        value, labels, metric="sqeuclidean"), 3))
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs


[0.212, 0.012, 0.448, 0.125, 0.143, 0.271]

In [11]:
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(list(agrupamiento[key]), 0)],
    "cluster1": df_ml.iloc[find_indexs(list(agrupamiento[key]), 1)],
    "cluster2": df_ml.iloc[find_indexs(list(agrupamiento[key]), 2)],
    "cluster3": df_ml.iloc[find_indexs(list(agrupamiento[key]), 3)],

}
# preprocesado vs sentimiento.
print("rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rsScore cluster 1 cluster 2 cluster 3 cluster 4
0.212 & anger (56), sadness (8) &  &  & anger (20), sadness (12), joy (4)
0.012 & anger (2), sadness (1) & anger (26), sadness (8), joy (2) & anger (8), sadness (3), joy (1) & anger (40), sadness (8), joy (1)
0.448 & anger (22), sadness (3) & anger (26), sadness (7), joy (1) & anger (21), sadness (7), joy (2) & anger (7), sadness (3), joy (1)
0.125 &  & anger (42), sadness (7), joy (2) &  & anger (34), sadness (13), joy (2)
0.143 &  &  & anger (27), sadness (12), joy (3) & anger (49), sadness (8), joy (1)
0.271 & anger (41), sadness (12) &  & anger (35), sadness (8), joy (4) & 


# eugenspace fuzzy c-means

In [12]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=3, n_iter=2)

agrupamiento = {}
metric = []
rs = []
for key, value in data_ml.items():
    data = svd.fit_transform(np.array(value))
    fcm = FCM(n_clusters=4)
    fcm.fit(data)
    labels = fcm.predict(data)
    grupos = fcm.soft_predict(data)
    agrupamiento[key] = labels
    rs.append(round(metrics.silhouette_score(
        value, labels, metric="sqeuclidean"), 3))
    #print(f"{round(metrica[0], 3)} {round(metrica[1], 3)} {round(metrica[2], 3)} {round(metrica[3], 3)} {round(metrica[4], 3)} {round(metrica[5], 3)}")
rs

[0.017, 0.031, 0.452, 0.046, 0.023, 0.063]

In [13]:
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(list(agrupamiento[key]), 0)],
    "cluster1": df_ml.iloc[find_indexs(list(agrupamiento[key]), 1)],
    "cluster2": df_ml.iloc[find_indexs(list(agrupamiento[key]), 2)],
    "cluster3": df_ml.iloc[find_indexs(list(agrupamiento[key]), 3)],

}
# preprocesado vs sentimiento.
print("rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i = 0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

rsScore cluster 1 cluster 2 cluster 3 cluster 4
0.017 & anger (39), sadness (5) & anger (9), sadness (6), joy (1) & joy (1), sadness (1) & anger (28), sadness (8), joy (2)
0.031 & joy (1), sadness (1) & anger (27), sadness (8) & anger (15), sadness (9), joy (3) & anger (34), sadness (2)
0.452 & anger (20), sadness (7), joy (2) & anger (20), sadness (3) & anger (29), sadness (7), joy (1) & anger (7), sadness (3), joy (1)
0.046 & anger (22), sadness (5) & anger (17), sadness (7), joy (2) & anger (12), sadness (8), joy (2) & anger (25)
0.023 & anger (18), sadness (8), joy (2) & anger (25), sadness (3) & anger (21), sadness (1) & anger (12), sadness (8), joy (2)
0.063 & anger (26), sadness (4) & anger (17), sadness (3), joy (1) & anger (21), sadness (8) & anger (12), sadness (5), joy (3)


# Extracción de infrmación

In [20]:
import openai
import os
import ast
from config import OPENAI_API_KEY

def ei(context, t):
    response = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"{context}:{t!r}",
            temperature=0.7,
            max_tokens=64,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    return response


openai.api_key = OPENAI_API_KEY
context_e = """
        Extrae eventos especificando el tipo de evento,
        tipos de evento:[Fecha, lugar, concecuencias],
        el resultado retornalo como un diccionario de python, ejemplo:
        {
          "tipo evento": "algo",
          "lugar": "algo",
          "consecuencias": "algo",
        }
        es imperativo que respetes el formato de respuesta.
    """
context_s ="""
        Escribe un resumen de este texto:
    """
context = [context_e, context_s]
info_extraction = {}
for key, value in clusters["max_polling_max"].items():
    texts = list(value["tweet_text"])
    t = ""
    info_extraction[key] = {
        "resumen": "",
        "eventos": "",
    }
    for text in texts:
        # info_extraction[key] = {
        #     "tipo evento": [],
        #     "lugar": [],
        #     "consecuencias": [],
        # }
        t += str(text)
    info_extraction[key]["resumen"] = ei(context_s, t).choices[0]["text"]
    info_extraction[key]["eventos"] = ei(context_e, t).choices[0]["text"]


In [21]:
info_extraction

{'cluster0': {'resumen': '\n\nEste texto muestra el descontento de un cliente por el retraso en la entrega de un regalo para navidad. El pedido debía llegar el 17 de diciembre, pero aún no se ha recibido. Al',
  'eventos': '\n\nEvento:\n{\n  "tipo evento": "Retraso en la entrega de una compra",\n  "lugar": "Varios lugares (callcenter, tiendas, etc)",\n  "consecuencias": "Dejar a'},
 'cluster1': {'resumen': '\n\nEste texto se refiere a una mala experiencia de alguien que agradece la lección aprendida a pesar de la situación negativa. Esta experiencia ha enseñado a esa persona a valorar y agradecer',
  'eventos': '\n    Respuesta:\n    {\n      "tipo evento": "mala experiencia",\n      "lugar": "",\n      "consecuencias": "",\n    }'},
 'cluster2': {'resumen': '\n\nEste texto relata la experiencia de una persona al comprar por internet para navidad. Esta compra no se entregó a tiempo, lo que provocó que su hijo no recibiera su regalo a tiempo, y que el usu',
  'eventos': '\n\nRespuesta:\